#Homework 2 - Spark

## Lettura dei dati

In [0]:
import requests

from pyspark.sql.functions import isnan, when, count, isnull

In [0]:
local_path_popolazione_regioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/popolazione_regioni.csv"
local_path_dati_covid_regioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_covid_regioni.csv"
local_path_dati_vaccini_somministrazioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_vaccini_somministrazioni.csv"
local_path_dati_vaccini_consegne = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_vaccini_consegne.csv"

# local_path_popolazione_regioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/popolazione_regioni.csv"
# local_path_dati_covid_regioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_covid_regioni.csv"
# local_path_dati_vaccini_somministrazioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_vaccini_somministrazioni.csv"
# local_path_dati_vaccini_consegne = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_vaccini_consegne.csv"

popolazione_regioni_URL = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-statistici-riferimento/popolazione-istat-regione-range.csv"
dati_covid_regioni_URL = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv"
dati_vaccini_somministrazioni_URL = "https://raw.githubusercontent.com/italia/covid19-opendata-vaccini/master/dati/somministrazioni-vaccini-latest.csv"
dati_vaccini_consegne_URL = "https://raw.githubusercontent.com/italia/covid19-opendata-vaccini/master/dati/consegne-vaccini-latest.csv"

dbutils.fs.put(local_path_popolazione_regioni, requests.get(popolazione_regioni_URL).text, True)
dbutils.fs.put(local_path_dati_covid_regioni, requests.get(dati_covid_regioni_URL).text, True)
dbutils.fs.put(local_path_dati_vaccini_somministrazioni, requests.get(dati_vaccini_somministrazioni_URL).text, True)
dbutils.fs.put(local_path_dati_vaccini_consegne, requests.get(dati_vaccini_consegne_URL).text, True)

popolazione_regioni = spark.read.option("inferSchema", "true") \
                                .option("header", "true") \
                                .csv(local_path_popolazione_regioni)
dati_covid_regioni = spark.read.option("inferSchema", "true") \
                                .option("header", "true") \
                                .csv(local_path_dati_covid_regioni)
dati_vaccini_somministrazioni = spark.read.option("inferSchema", "true") \
                                          .option("header", "true") \
                                          .csv(local_path_dati_vaccini_somministrazioni)
dati_vaccini_consegne = spark.read.option("inferSchema", "true") \
                                  .option("header", "true") \
                                  .csv(local_path_dati_vaccini_consegne)

Wrote 17300 bytes.
Wrote 1485774 bytes.
Wrote 3798235 bytes.
Wrote 55099 bytes.

## Preprocessing

### Popolazione Regioni
Si eliminano le colonne che non sono di interesse.

In [0]:
popolazione_regioni = popolazione_regioni.drop('codice_regione', 'latitudine_regione', 'longitudine_regione')
popolazione_regioni.show()

# popolazione_regioni.groupBy('denominazione_regione').sum('totale_generale').orderBy('sum(totale_generale)', ascending=False).show()

+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
codice_nuts_1|descrizione_nuts_1|codice_nuts_2|denominazione_regione|sigla_regione|range_eta|totale_genere_maschile|totale_genere_femminile|totale_generale|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
 ITF| Sud| ITF1| Abruzzo| ABR| 0-15| 87423| 82474| 169897|
 ITF| Sud| ITF1| Abruzzo| ABR| 16-19| 24402| 22349| 46751|
 ITF| Sud| ITF1| Abruzzo| ABR| 20-29| 67472| 62611| 130083|
 ITF| Sud| ITF1| Abruzzo| ABR| 30-39| 76472| 74241| 150713|
 ITF| Sud| ITF1| Abruzzo| ABR| 40-49| 94745| 95315| 190060|
 ITF| Sud| ITF1| Abruzzo| ABR| 50-59| 100048| 104109| 204157|
 ITF| Sud| ITF1| Abruzzo| ABR| 60-69| 80545| 87160| 167705|
 ITF| Sud| ITF1| Abruzzo| ABR| 70-79| 60810| 69762| 130572|
 ITF| Sud| ITF1| Abruzzo| ABR| 80-89| 34181| 50307| 84488|
 ITF| Sud| ITF1| Abruzzo| ABR| 90+| 5645| 13870| 19515|
 ITF| Sud| ITF5| Basilicata| BAS| 0-15| 36113| 33881| 69994|
 ITF| Sud| ITF5| Basilicata| BAS| 16-19| 11598| 10434| 22032|
 ITF| Sud| ITF5| Basilicata| BAS| 20-29| 32068| 29115| 61183|
 ITF| Sud| ITF5| Basilicata| BAS| 30-39| 33556| 31648| 65204|
 ITF| Sud| ITF5| Basilicata| BAS| 40-49| 39260| 39207| 78467|
 ITF| Sud| ITF5| Basilicata| BAS| 50-59| 43150| 44672| 87822|
 ITF| Sud| ITF5| Basilicata| BAS| 60-69| 35360| 37457| 72817|
 ITF| Sud| ITF5| Basilicata| BAS| 70-79| 24050| 27755| 51805|
 ITF| Sud| ITF5| Basilicata| BAS| 80-89| 14507| 21600| 36107|
 ITF| Sud| ITF5| Basilicata| BAS| 90+| 2488| 5335| 7823|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
only showing top 20 rows

Si verifica che il formato dei dati sia corretto.

In [0]:
popolazione_regioni.dtypes

Out[320]: [('codice_nuts_1', 'string'),
 ('descrizione_nuts_1', 'string'),
 ('codice_nuts_2', 'string'),
 ('denominazione_regione', 'string'),
 ('sigla_regione', 'string'),
 ('range_eta', 'string'),
 ('totale_genere_maschile', 'int'),
 ('totale_genere_femminile', 'int'),
 ('totale_generale', 'int')]

Si verifica la correttezza delle colonne categoriche.

In [0]:
num_regioni = popolazione_regioni.groupBy('denominazione_regione') \
                .count() \
                .count()
print('Il numero di regioni è:', num_regioni)
print()
popolazione_regioni.groupBy('codice_nuts_1', 'descrizione_nuts_1', 'codice_nuts_2', 'denominazione_regione', 'sigla_regione') \
                    .count() \
                    .drop('count') \
                    .orderBy('descrizione_nuts_1') \
                    .show(num_regioni)

Il numero di regioni è: 21

+-------------+------------------+-------------+---------------------+-------------+
codice_nuts_1|descrizione_nuts_1|codice_nuts_2|denominazione_regione|sigla_regione|
+-------------+------------------+-------------+---------------------+-------------+
 ITI| Centro| ITI4| Lazio| LAZ|
 ITI| Centro| ITI3| Marche| MAR|
 ITI| Centro| ITI2| Umbria| UMB|
 ITI| Centro| ITI1| Toscana| TOS|
 ITG| Isole| ITG1| Sicilia| SIC|
 ITG| Isole| ITG2| Sardegna| SAR|
 ITH| Nord-Est| ITH5| Emilia-Romagna| EMR|
 ITH| Nord-Est| ITH1| Bolzano| BOL|
 ITH| Nord-Est| ITH4| Friuli Venezia Gi...| FVG|
 ITH| Nord-Est| ITH2| Trento| TRE|
 ITH| Nord-Est| ITH3| Veneto| VEN|
 ITC| Nord-Ovest| ITC1| Piemonte| PIE|
 ITC| Nord-Ovest| ITC2| Valle d'Aosta| VDA|
 ITC| Nord-Ovest| ITC3| Liguria| LIG|
 ITC| Nord-Ovest| ITC4| Lombardia| LOM|
 ITF| Sud| ITF1| Abruzzo| ABR|
 ITF| Sud| ITF2| Molise| MOL|
 ITF| Sud| ITF3| Campania| CAM|
 ITF| Sud| ITF5| Basilicata| BAS|
 ITF| Sud| ITF4| Puglia| PUG|
 ITF| Sud| ITF6| Calabria| CAL|
+-------------+------------------+-------------+---------------------+-------------+

In [0]:
popolazione_regioni.groupBy('range_eta') \
                    .count() \
                    .orderBy('range_eta') \
                    .show()

+---------+-----+
range_eta|count|
+---------+-----+
 0-15| 21|
 16-19| 21|
 20-29| 21|
 30-39| 21|
 40-49| 21|
 50-59| 21|
 60-69| 21|
 70-79| 21|
 80-89| 21|
 90+| 21|
+---------+-----+

Si verifica la correttezza delle colonne numeriche.

In [0]:
popolazione_regioni.describe(['totale_genere_maschile', 'totale_genere_femminile', 'totale_generale']) \
                    .show()

+-------+----------------------+-----------------------+-----------------+
summary|totale_genere_maschile|totale_genere_femminile| totale_generale|
+-------+----------------------+-----------------------+-----------------+
 count| 210| 210| 210|
 mean| 138333.79047619048| 145673.29523809525|284007.0857142857|
 stddev| 151728.9786894938| 152578.20019731886|303588.4857882117|
 min| 457| 1307| 1764|
 max| 795710| 796399| 1592109|
+-------+----------------------+-----------------------+-----------------+

Si verifica la presenza di *missing values*.

In [0]:
popolazione_regioni.select([count(when(isnull(c), c)).alias(c) for c in popolazione_regioni.columns]).show()

+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
codice_nuts_1|descrizione_nuts_1|codice_nuts_2|denominazione_regione|sigla_regione|range_eta|totale_genere_maschile|totale_genere_femminile|totale_generale|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+

### Dati Covid Regioni
Si eliminano le colonne che non sono di interesse.

In [0]:
dati_covid_regioni = dati_covid_regioni.drop('stato', 'codice_regione', 'lat', 'long', 'note_test', 'note_casi', 'note',
                                             'casi_da_sospetto_diagnostico', 'casi_da_screening',
                                             'casi_testati', 'ingressi_terapia_intensiva', 
                                             'totale_positivi_test_molecolare', 'totale_positivi_test_antigenico_rapido',
                                             'tamponi_test_molecolare', 'tamponi_test_antigenico_rapido')
dati_covid_regioni.show()

+-------------------+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
 data|denominazione_regione|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati|isolamento_domiciliare|totale_positivi|variazione_totale_positivi|nuovi_positivi|dimessi_guariti|deceduti|totale_casi|tamponi|codice_nuts_1|codice_nuts_2|
+-------------------+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
2020-02-24 18:00:00| Abruzzo| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 5| null| null|
2020-02-24 18:00:00| Basilicata| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| null| null|
2020-02-24 18:00:00| Calabria| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| null| null|
2020-02-24 18:00:00| Campania| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 10| null| null|
2020-02-24 18:00:00| Emilia-Romagna| 10| 2| 12| 6| 18| 0| 18| 0| 0| 18| 148| null| null|
2020-02-24 18:00:00| Friuli Venezia Gi...| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 58| null| null|
2020-02-24 18:00:00| Lazio| 1| 1| 2| 0| 2| 0| 2| 1| 0| 3| 124| null| null|
2020-02-24 18:00:00| Liguria| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| null| null|
2020-02-24 18:00:00| Lombardia| 76| 19| 95| 71| 166| 0| 166| 0| 6| 172| 1463| null| null|
2020-02-24 18:00:00| Marche| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 16| null| null|
2020-02-24 18:00:00| Molise| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| null| null|
2020-02-24 18:00:00| P.A. Bolzano| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| null| null|
2020-02-24 18:00:00| P.A. Trento| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 3| null| null|
2020-02-24 18:00:00| Piemonte| 2| 0| 2| 1| 3| 0| 3| 0| 0| 3| 141| null| null|
2020-02-24 18:00:00| Puglia| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| null| null|
2020-02-24 18:00:00| Sardegna| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| null| null|
2020-02-24 18:00:00| Sicilia| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 5| null| null|
2020-02-24 18:00:00| Toscana| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 140| null| null|
2020-02-24 18:00:00| Umbria| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| null| null|
2020-02-24 18:00:00| Valle d'Aosta| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 7| null| null|
+-------------------+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
only showing top 20 rows

Si verifica che il formato dei dati sia corretto.

In [0]:
dati_covid_regioni.dtypes

Out[326]: [('data', 'timestamp'),
 ('denominazione_regione', 'string'),
 ('ricoverati_con_sintomi', 'int'),
 ('terapia_intensiva', 'int'),
 ('totale_ospedalizzati', 'int'),
 ('isolamento_domiciliare', 'int'),
 ('totale_positivi', 'int'),
 ('variazione_totale_positivi', 'int'),
 ('nuovi_positivi', 'int'),
 ('dimessi_guariti', 'int'),
 ('deceduti', 'int'),
 ('totale_casi', 'int'),
 ('tamponi', 'int'),
 ('codice_nuts_1', 'string'),
 ('codice_nuts_2', 'string')]

Si verifica la correttezza delle colonne categoriche.

In [0]:
num_regioni = dati_covid_regioni.groupBy('denominazione_regione') \
                .count() \
                .count()
print('Il numero di regioni è:', num_regioni)
print()
dati_covid_regioni.groupBy('codice_nuts_1', 'codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .orderBy('codice_nuts_1') \
                    .show(50)

Il numero di regioni è: 21

+-------------+-------------+---------------------+
codice_nuts_1|codice_nuts_2|denominazione_regione|
+-------------+-------------+---------------------+
 null| null| Lazio|
 null| null| Sicilia|
 null| null| Emilia-Romagna|
 null| null| Toscana|
 null| null| Puglia|
 null| null| Abruzzo|
 null| null| Sardegna|
 null| null| Umbria|
 null| null| Piemonte|
 null| null| P.A. Bolzano|
 null| null| Molise|
 null| null| Calabria|
 null| null| Friuli Venezia Gi...|
 null| null| Valle d'Aosta|
 null| null| P.A. Trento|
 null| null| Marche|
 null| null| Basilicata|
 null| null| Veneto|
 null| null| Campania|
 null| null| Lombardia|
 null| null| Liguria|
 1| null| P.A. Bolzano|
 172410| ITI| Umbria|
 500340| ITH| P.A. Bolzano|
 ITC| ITC2| Valle d'Aosta|
 ITC| ITC3| Liguria|
 ITC| ITC1| Piemonte|
 ITC| ITC4| Lombardia|
 ITF| ITF6| Calabria|
 ITF| ITF2| Molise|
 ITF| ITF5| Basilicata|
 ITF| ITF1| Abruzzo|
 ITF| ITF3| Campania|
 ITF| ITF4| Puglia|
 ITG| ITG1| Sicilia|
 ITG| ITG2| Sardegna|
 ITH| ITH3| Veneto|
 ITH| ITH5| Emilia-Romagna|
 ITH| ITH2| P.A. Trento|
 ITH| ITH1| P.A. Bolzano|
 ITH| ITH4| Friuli Venezia Gi...|
 ITI| ITI1| Toscana|
 ITI| ITI2| Umbria|
 ITI| ITI3| Marche|
 ITI| ITI4| Lazio|
+-------------+-------------+---------------------+

Si possono fare alcune osservazioni:
- per alcune entry mancano i codici nuts;
- per aclune entry i codici nuts sono sbagliati (problema dovuto alla lettura del file csv);
- alcune regioni (Bolzano e Trento) hanno una denominazione diversa rispetto alla tabella `popolazione_regioni`;

In [0]:
join1 = popolazione_regioni.join(dati_covid_regioni, on='denominazione_regione', how='anti') \
                    .groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .withColumnRenamed('denominazione_regione', 'popolazione_regioni_denominazione_regione')

join2 = dati_covid_regioni.join(popolazione_regioni, on='denominazione_regione', how='anti') \
                    .groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .withColumnRenamed('denominazione_regione', 'dati_covid_regioni_denominazione_regione')

join1.join(join2, on='codice_nuts_2').show()

dati_covid_regioni = dati_covid_regioni.replace('P.A. Trento', value='Trento', subset='denominazione_regione')
dati_covid_regioni = dati_covid_regioni.replace('P.A. Bolzano', value='Bolzano', subset='denominazione_regione')

+-------------+-----------------------------------------+----------------------------------------+
codice_nuts_2|popolazione_regioni_denominazione_regione|dati_covid_regioni_denominazione_regione|
+-------------+-----------------------------------------+----------------------------------------+
 ITH1| Bolzano| P.A. Bolzano|
 ITH2| Trento| P.A. Trento|
+-------------+-----------------------------------------+----------------------------------------+

In [0]:
dati_covid_regioni = dati_covid_regioni.drop('codice_nuts_1', 'codice_nuts_2')

data = popolazione_regioni.groupBy('codice_nuts_1', 'codice_nuts_2', 'denominazione_regione') \
                          .count() \
                          .drop('count')

dati_covid_regioni = dati_covid_regioni.join(data, on=['denominazione_regione'], how='left')

dati_covid_regioni.groupBy('codice_nuts_1', 'codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .orderBy('codice_nuts_1').show(num_regioni)

+-------------+-------------+---------------------+
codice_nuts_1|codice_nuts_2|denominazione_regione|
+-------------+-------------+---------------------+
 ITC| ITC2| Valle d'Aosta|
 ITC| ITC3| Liguria|
 ITC| ITC1| Piemonte|
 ITC| ITC4| Lombardia|
 ITF| ITF4| Puglia|
 ITF| ITF1| Abruzzo|
 ITF| ITF6| Calabria|
 ITF| ITF2| Molise|
 ITF| ITF5| Basilicata|
 ITF| ITF3| Campania|
 ITG| ITG2| Sardegna|
 ITG| ITG1| Sicilia|
 ITH| ITH3| Veneto|
 ITH| ITH5| Emilia-Romagna|
 ITH| ITH1| Bolzano|
 ITH| ITH2| Trento|
 ITH| ITH4| Friuli Venezia Gi...|
 ITI| ITI1| Toscana|
 ITI| ITI2| Umbria|
 ITI| ITI3| Marche|
 ITI| ITI4| Lazio|
+-------------+-------------+---------------------+

Si verifica la correttezza delle colonne numeriche.

In [0]:
dati_covid_regioni.describe(['ricoverati_con_sintomi', 'terapia_intensiva', 'totale_ospedalizzati', 'dimessi_guariti']) \
                  .show()

+-------+----------------------+------------------+--------------------+-----------------+
summary|ricoverati_con_sintomi| terapia_intensiva|totale_ospedalizzati| dimessi_guariti|
+-------+----------------------+------------------+--------------------+-----------------+
 count| 8988| 8988| 8988| 8988|
 mean| 710.4000890075656| 81.29372496662216| 791.6938139741878| 40867.8722741433|
 stddev| 1292.5532118318006|143.00589322648972| 1430.8251679041118|84973.39642601715|
 min| 0| 0| 0| 0|
 max| 12077| 1381| 13328| 710637|
+-------+----------------------+------------------+--------------------+-----------------+

In [0]:
dati_covid_regioni.describe(['isolamento_domiciliare', 'totale_positivi', 'variazione_totale_positivi', 'nuovi_positivi', 'deceduti', 'totale_casi', 'tamponi']) \
                  .show()

+-------+----------------------+------------------+--------------------------+------------------+------------------+------------------+------------------+
summary|isolamento_domiciliare| totale_positivi|variazione_totale_positivi| nuovi_positivi| deceduti| totale_casi| tamponi|
+-------+----------------------+------------------+--------------------------+------------------+------------------+------------------+------------------+
 count| 8988| 8988| 8988| 8988| 8988| 8988| 8988|
 mean| 11597.211059190031| 12388.90487316422| 50.35502892745883|441.12505562972854|2439.7059412550066| 55696.48509123275| 830756.6902536716|
 stddev| 21211.774938890743|22216.134347624353| 828.2009975915025| 855.761383377797| 4656.140807335724|104567.67922190507|1309806.7926228403|
 min| 0| 0| -21926| -229| 0| 0| 0|
 max| 155066| 164406| 10263| 11489| 32688| 796289| 9277607|
+-------+----------------------+------------------+--------------------------+------------------+------------------+------------------+------------------+

In [0]:
dati_covid_regioni.filter(condition='nuovi_positivi < 0').show()

+---------------------+-------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
denominazione_regione| data|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati|isolamento_domiciliare|totale_positivi|variazione_totale_positivi|nuovi_positivi|dimessi_guariti|deceduti|totale_casi|tamponi|codice_nuts_1|codice_nuts_2|
+---------------------+-------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
 Piemonte|2020-02-27 18:00:00| 2| 0| 2| 0| 2| -1| -1| 0| 0| 2| 156| ITC| ITC1|
 Liguria|2020-03-01 17:00:00| 12| 1| 13| 8| 21| -17| -17| 4| 0| 25| 121| ITC| ITC3|
 Liguria|2020-03-02 18:00:00| 12| 1| 13| 5| 18| -3| -3| 4| 0| 22| 121| ITC| ITC3|
 Sicilia|2020-03-02 18:00:00| 2| 0| 2| 3| 5| -2| -2| 2| 0| 7| 307| ITG| ITG1|
 Piemonte|2020-03-09 18:00:00| 222| 50| 272| 65| 337| -18| -10| 0| 13| 350| 1681| ITC| ITC1|
 Calabria|2020-04-17 17:00:00| 154| 7| 161| 658| 819| -28| -18| 99| 73| 991| 21657| ITF| ITF6|
 Sardegna|2020-05-04 17:00:00| 91| 9| 100| 553| 653| -36| -2| 545| 119| 1317| 28052| ITG| ITG2|
 Basilicata|2020-05-07 17:00:00| 48| 2| 50| 105| 155| -17| -16| 202| 26| 383| 16272| ITF| ITF5|
 Basilicata|2020-05-08 17:00:00| 48| 2| 50| 102| 152| -3| -1| 204| 26| 382| 16777| ITF| ITF5|
 Marche|2020-05-19 17:00:00| 144| 17| 161| 1967| 2128| -187| -3| 3561| 986| 6675| 89985| ITI| ITI3|
 Sardegna|2020-05-25 17:00:00| 49| 3| 52| 179| 231| -14| -2| 994| 129| 1354| 51073| ITG| ITG2|
 Sardegna|2020-06-09 17:00:00| 9| 1| 10| 44| 54| -2| -1| 1176| 131| 1361| 64272| ITG| ITG2|
 Campania|2020-06-12 17:00:00| 61| 2| 63| 283| 346| -264| -229| 3832| 430| 4608| 230551| ITF| ITF3|
+---------------------+-------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+

Si verifica la presenza di *missing values*.

In [0]:
dati_covid_regioni.select([count(when(isnull(c), c)).alias(c) for c in dati_covid_regioni.drop('data').columns]).show()

+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
denominazione_regione|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati|isolamento_domiciliare|totale_positivi|variazione_totale_positivi|nuovi_positivi|dimessi_guariti|deceduti|totale_casi|tamponi|codice_nuts_1|codice_nuts_2|
+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+

### Dati Somministrazioni Vaccini

Si elmininano le colonne che non sono di interesse

In [0]:
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.drop('area', 'codice_regione_ISTAT')

dati_vaccini_somministrazioni.show()

+---------------------+---------------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+----------+
data_somministrazione| fornitore|fascia_anagrafica|sesso_maschile|sesso_femminile|categoria_operatori_sanitari_sociosanitari|categoria_personale_non_sanitario|categoria_ospiti_rsa|categoria_60_69|categoria_70_79|categoria_over80|categoria_forze_armate|categoria_personale_scolastico|categoria_soggetti_fragili|categoria_altro|prima_dose|seconda_dose|codice_NUTS1|codice_NUTS2| nome_area|
+---------------------+---------------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+----------+
 2020-12-27|Pfizer/BioNTech| 20-29| 1| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 30-39| 1| 3| 4| 0| 0| 0| 0| 0| 0| 0| 0| 0| 4| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 40-49| 1| 6| 7| 0| 0| 0| 0| 0| 0| 0| 0| 0| 7| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 50-59| 4| 5| 9| 0| 0| 0| 0| 0| 0| 0| 0| 0| 9| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 60-69| 10| 4| 13| 1| 0| 0| 0| 0| 0| 0| 0| 0| 14| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 70-79| 1| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 1| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 80-89| 1| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 1| 0| ITF| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 20-29| 4| 5| 9| 0| 0| 0| 0| 0| 0| 0| 0| 0| 9| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 30-39| 10| 18| 27| 0| 0| 0| 0| 0| 0| 0| 0| 1| 28| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 40-49| 7| 23| 29| 0| 0| 0| 0| 0| 0| 0| 0| 1| 30| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 50-59| 11| 15| 26| 0| 0| 0| 0| 0| 0| 0| 0| 0| 26| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 60-69| 9| 4| 13| 0| 0| 0| 0| 0| 0| 0| 0| 0| 13| 0| ITF| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 20-29| 15| 34| 49| 0| 0| 0| 0| 0| 0| 0| 0| 0| 49| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 30-39| 23| 20| 43| 0| 0| 0| 0| 0| 0| 0| 0| 0| 43| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 40-49| 22| 27| 49| 0| 0| 0| 0| 0| 0| 0| 0| 0| 49| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 50-59| 47| 28| 74| 1| 0| 0| 0| 0| 0| 0| 0| 0| 75| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 60-69| 43| 16| 59| 0| 0| 0| 0| 0| 0| 0| 0| 0| 59| 0| ITF| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 20-29| 27| 33| 59| 1| 0| 0| 0| 0| 0| 0| 0| 0| 60| 0| ITF| ITF3| Campania|
 2020-12-27|Pfizer/BioNTech| 30-39| 61| 64| 122| 2| 0| 0| 0| 0| 0| 0| 0| 1| 125| 0| ITF| ITF3| Campania|
 2020-12-27|Pfizer/BioNTech| 40-49| 80| 87| 164| 3| 0| 0| 0| 0| 0| 0| 0| 0| 167| 0| ITF| ITF3| Campania|
+---------------------+---------------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+----------+
only showing top 20 rows

Si verifica che il formato dei dati sia corretto

In [0]:
dati_vaccini_somministrazioni.dtypes

Out[335]: [('data_somministrazione', 'string'),
 ('fornitore', 'string'),
 ('fascia_anagrafica', 'string'),
 ('sesso_maschile', 'int'),
 ('sesso_femminile', 'int'),
 ('categoria_operatori_sanitari_sociosanitari', 'int'),
 ('categoria_personale_non_sanitario', 'int'),
 ('categoria_ospiti_rsa', 'int'),
 ('categoria_60_69', 'int'),
 ('categoria_70_79', 'int'),
 ('categoria_over80', 'int'),
 ('categoria_forze_armate', 'int'),
 ('categoria_personale_scolastico', 'int'),
 ('categoria_soggetti_fragili', 'int'),
 ('categoria_altro', 'int'),
 ('prima_dose', 'int'),
 ('seconda_dose', 'int'),
 ('codice_NUTS1', 'string'),
 ('codice_NUTS2', 'string'),
 ('nome_area', 'string')]

Si verifica la correttezza delle colonne categoriche.

In [0]:
num_regioni = dati_vaccini_somministrazioni.groupBy('nome_area') \
                                           .count() \
                                           .count()
print('Il numero di regioni è:', num_regioni)
print()
dati_vaccini_somministrazioni.groupBy('codice_NUTS1', 'codice_NUTS2', 'nome_area') \
                             .count() \
                             .drop('count') \
                             .orderBy('codice_NUTS1') \
                             .show(50)

Il numero di regioni è: 21

+------------+------------+--------------------+
codice_NUTS1|codice_NUTS2| nome_area|
+------------+------------+--------------------+
 ITC| ITC2|Valle d'Aosta / V...|
 ITC| ITC3| Liguria|
 ITC| ITC1| Piemonte|
 ITC| ITC4| Lombardia|
 ITF| ITF4| Puglia|
 ITF| ITF1| Abruzzo|
 ITF| ITF6| Calabria|
 ITF| ITF2| Molise|
 ITF| ITF5| Basilicata|
 ITF| ITF3| Campania|
 ITG| ITG2| Sardegna|
 ITG| ITG1| Sicilia|
 ITH| ITH5| Emilia-Romagna|
 ITH| ITH1|Provincia Autonom...|
 ITH| ITH4|Friuli-Venezia Gi...|
 ITH| ITH3| Veneto|
 ITH| ITH2|Provincia Autonom...|
 ITI| ITI1| Toscana|
 ITI| ITI2| Umbria|
 ITI| ITI3| Marche|
 ITI| ITI4| Lazio|
+------------+------------+--------------------+

In [0]:
join1 = popolazione_regioni.join(dati_vaccini_somministrazioni, popolazione_regioni.denominazione_regione == dati_vaccini_somministrazioni.nome_area, how='anti') \
                    .groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') 

join2 = dati_vaccini_somministrazioni.join(popolazione_regioni, popolazione_regioni.denominazione_regione == dati_vaccini_somministrazioni.nome_area, how='anti') \
                    .groupBy('codice_NUTS2', 'nome_area') \
                    .count() \
                    .drop('count') 

join1.join(join2, popolazione_regioni.codice_nuts_2 == dati_vaccini_somministrazioni.codice_NUTS2).show()

dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.replace('Provincia Autonoma Trento', value='Trento', subset='nome_area')
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.replace('Provincia Autonoma Bolzano / Bozen', value='Bolzano', subset='nome_area')
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.replace('Friuli-Venezia Giulia', value='Friuli Venezia Giulia', subset='nome_area')
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.replace('Valle d\'Aosta / Vallée d\'Aoste', value='Valle d\'Aosta', subset='nome_area')

+-------------+---------------------+------------+--------------------+
codice_nuts_2|denominazione_regione|codice_NUTS2| nome_area|
+-------------+---------------------+------------+--------------------+
 ITC2| Valle d'Aosta| ITC2|Valle d'Aosta / V...|
 ITH2| Trento| ITH2|Provincia Autonom...|
 ITH4| Friuli Venezia Gi...| ITH4|Friuli-Venezia Gi...|
 ITH1| Bolzano| ITH1|Provincia Autonom...|
+-------------+---------------------+------------+--------------------+

In [0]:
dati_vaccini_somministrazioni.groupBy('codice_NUTS1', 'codice_NUTS2', 'nome_area') \
                             .count() \
                             .drop('count') \
                             .orderBy('codice_NUTS1') \
                             .show(num_regioni)

+------------+------------+--------------------+
codice_NUTS1|codice_NUTS2| nome_area|
+------------+------------+--------------------+
 ITC| ITC2| Valle d'Aosta|
 ITC| ITC3| Liguria|
 ITC| ITC1| Piemonte|
 ITC| ITC4| Lombardia|
 ITF| ITF4| Puglia|
 ITF| ITF1| Abruzzo|
 ITF| ITF6| Calabria|
 ITF| ITF2| Molise|
 ITF| ITF5| Basilicata|
 ITF| ITF3| Campania|
 ITG| ITG2| Sardegna|
 ITG| ITG1| Sicilia|
 ITH| ITH3| Veneto|
 ITH| ITH5| Emilia-Romagna|
 ITH| ITH1| Bolzano|
 ITH| ITH2| Trento|
 ITH| ITH4|Friuli Venezia Gi...|
 ITI| ITI1| Toscana|
 ITI| ITI2| Umbria|
 ITI| ITI3| Marche|
 ITI| ITI4| Lazio|
+------------+------------+--------------------+

In [0]:
dati_vaccini_somministrazioni.groupBy('fascia_anagrafica') \
                    .count() \
                    .drop('count') \
                    .orderBy('fascia_anagrafica') \
                    .show()

+-----------------+
fascia_anagrafica|
+-----------------+
 16-19|
 20-29|
 30-39|
 40-49|
 50-59|
 60-69|
 70-79|
 80-89|
 90+|
+-----------------+

Si osserva che rispetto alla tabella popolazione_regioni manca la `fascia_anagrafica` 0-15

Si verifica la correttezza delle colonne numeriche.

In [0]:
dati_vaccini_somministrazioni.describe(['sesso_maschile', 'sesso_femminile', 'prima_dose', 'seconda_dose']) \
                             .show()

+-------+------------------+------------------+-----------------+------------------+
summary| sesso_maschile| sesso_femminile| prima_dose| seconda_dose|
+-------+------------------+------------------+-----------------+------------------+
 count| 40429| 40429| 40429| 40429|
 mean|188.11721783868015|254.16713250389571|312.6598233940983|129.62452694847758|
 stddev| 529.8966389255627| 684.4111107795999|931.9893283225084| 509.5584847877542|
 min| 0| 0| 0| 0|
 max| 13117| 18407| 26492| 15060|
+-------+------------------+------------------+-----------------+------------------+

In [0]:
dati_vaccini_somministrazioni.describe(['categoria_operatori_sanitari_sociosanitari', 'categoria_personale_non_sanitario', 'categoria_ospiti_rsa', 'categoria_60_69', 'categoria_70_79', 'categoria_over80', 'categoria_forze_armate', 'categoria_personale_scolastico', 'categoria_soggetti_fragili', 'categoria_altro']) \
                             .show()

+-------+------------------------------------------+---------------------------------+--------------------+------------------+-----------------+-----------------+----------------------+------------------------------+--------------------------+-----------------+
summary|categoria_operatori_sanitari_sociosanitari|categoria_personale_non_sanitario|categoria_ospiti_rsa| categoria_60_69| categoria_70_79| categoria_over80|categoria_forze_armate|categoria_personale_scolastico|categoria_soggetti_fragili| categoria_altro|
+-------+------------------------------------------+---------------------------------+--------------------+------------------+-----------------+-----------------+----------------------+------------------------------+--------------------------+-----------------+
 count| 40429| 40429| 40429| 40429| 40429| 40429| 40429| 40429| 40429| 40429|
 mean| 78.8526552722056| 21.56573251873655| 15.833708476588587|16.999480570877342|61.10299537460734|143.1479878305177| 7.786292018105815| 28.444161369314106| 60.34381260976032|8.207524301862524|
 stddev| 232.26374213476564| 67.80146349970514| 62.467660381997575|209.73849809368198|626.8388838645126|900.1603775400728| 43.654453133638874| 155.71865582611935| 267.92555352419|37.47973327585045|
 min| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 max| 4661| 2303| 2284| 8627| 26100| 31002| 1214| 3583| 5780| 2008|
+-------+------------------------------------------+---------------------------------+--------------------+------------------+-----------------+-----------------+----------------------+------------------------------+--------------------------+-----------------+

Si verifica la presenza di *missing values*.

In [0]:
dati_vaccini_somministrazioni.select([count(when(isnull(c), c)).alias(c) for c in dati_vaccini_somministrazioni.drop('data').columns]).show()

+---------------------+---------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+---------+
data_somministrazione|fornitore|fascia_anagrafica|sesso_maschile|sesso_femminile|categoria_operatori_sanitari_sociosanitari|categoria_personale_non_sanitario|categoria_ospiti_rsa|categoria_60_69|categoria_70_79|categoria_over80|categoria_forze_armate|categoria_personale_scolastico|categoria_soggetti_fragili|categoria_altro|prima_dose|seconda_dose|codice_NUTS1|codice_NUTS2|nome_area|
+---------------------+---------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+---------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---------------------+---------+-----------------+--------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+------------+---------+

## Analisi

### Percentuale popolazione vaccinata per regione
Si riporta la percentuale di popolazione vaccinata (prima e seconda dose) per regione.

In [0]:
dosi_somministrate_regione = dati_vaccini_somministrazioni.groupBy('nome_area') \
                                                          .sum('prima_dose', 'seconda_dose') \
                                                          .orderBy('nome_area') \
                                                          .withColumnRenamed('nome_area', 'regione') \
                                                          .withColumnRenamed('sum(prima_dose)', 'vaccinati_prima_dose') \
                                                          .withColumnRenamed('sum(seconda_dose)', 'vaccinati_seconda_dose')

popolazione = popolazione_regioni.groupBy('denominazione_regione').sum('totale_generale')

cond = [popolazione.denominazione_regione == dosi_somministrate_regione.regione]
dosi_somministrate_regione = dosi_somministrate_regione.join(popolazione, cond, how='left') \
                                                       .drop('denominazione_regione') \
                                                       .withColumnRenamed('sum(totale_generale)', 'popolazione') \
                                                       .orderBy('regione')

dosi_somministrate_regione = dosi_somministrate_regione.withColumn('percentuale_prima_dose', dosi_somministrate_regione.vaccinati_prima_dose / dosi_somministrate_regione.popolazione * 100)
dosi_somministrate_regione = dosi_somministrate_regione.withColumn('percentuale_seconda_dose', dosi_somministrate_regione.vaccinati_seconda_dose / dosi_somministrate_regione.popolazione * 100)

dosi_somministrate_regione.show(21)

+--------------------+--------------------+----------------------+-----------+----------------------+------------------------+
 regione|vaccinati_prima_dose|vaccinati_seconda_dose|popolazione|percentuale_prima_dose|percentuale_seconda_dose|
+--------------------+--------------------+----------------------+-----------+----------------------+------------------------+
 Abruzzo| 279639| 112288| 1293941| 21.611418140394345| 8.677984544890377|
 Basilicata| 110879| 51618| 553254| 20.041246877564372| 9.329891876064158|
 Bolzano| 129635| 49059| 532644| 24.338019390061653| 9.21046702863451|
 Calabria| 336947| 147745| 1894110| 17.789199148940664| 7.800233354979383|
 Campania| 1116544| 424298| 5712143| 19.54684957992123| 7.428000314417899|
 Emilia-Romagna| 1011480| 468269| 4464119| 22.65799814028255| 10.489617324269357|
Friuli Venezia Gi...| 264396| 119867| 1206216| 21.919457211643685| 9.937440723717808|
 Lazio| 1217294| 533105| 5755700| 21.149364977326822| 9.262209635665513|
 Liguria| 382510| 156349| 1524826| 25.085485163553088| 10.253563357392908|
 Lombardia| 2055893| 842189| 10027602| 20.50233944267034| 8.398707886491707|
 Marche| 372367| 127837| 1512672| 24.6165064204269| 8.451072010323454|
 Molise| 72052| 34220| 300516| 23.976094450877824| 11.387080887540098|
 Piemonte| 992225| 417519| 4311217| 23.014963060314525| 9.684481203335391|
 Puglia| 873374| 287572| 3953305| 22.09224939639112| 7.274217395318601|
 Sardegna| 326456| 138394| 1611621| 20.256375413326086| 8.587254695738018|
 Sicilia| 895592| 401065| 4875290| 18.370025167733612| 8.226484988585295|
 Toscana| 825865| 329332| 3692555| 22.36567904878871| 8.918810958807654|
 Trento| 118438| 41511| 545425| 21.71480955218408| 7.610762249621854|
 Umbria| 199311| 78720| 870165| 22.904966299494923| 9.046560135146782|
 Valle d'Aosta| 29886| 10598| 125034| 23.902298574787658| 8.476094502295375|
 Veneto| 1029741| 469035| 4879133| 21.104999597264516| 9.61308084858519|
+--------------------+--------------------+----------------------+-----------+----------------------+------------------------+

### Percentuale popolazione vaccinata per fascia anagrafica
Si riporta la percentuale di popolazione vaccinata (prima e seconda dose) per fascia anagrafica.

In [0]:
dosi_somministrate_fascia = dati_vaccini_somministrazioni.groupBy('fascia_anagrafica') \
                                                          .sum('prima_dose', 'seconda_dose') \
                                                          .orderBy('fascia_anagrafica') \
                                                          .withColumnRenamed('sum(prima_dose)', 'vaccinati_prima_dose') \
                                                          .withColumnRenamed('sum(seconda_dose)', 'vaccinati_seconda_dose')

popolazione = popolazione_regioni.groupBy('range_eta').sum('totale_generale')

cond = [popolazione.range_eta == dosi_somministrate_fascia.fascia_anagrafica]
dosi_somministrate_fascia = dosi_somministrate_fascia.join(popolazione, cond, how='left') \
                                                       .withColumnRenamed('sum(totale_generale)', 'popolazione') \
                                                       .drop('range_eta') \
                                                       .orderBy('fascia_anagrafica')

dosi_somministrate_fascia = dosi_somministrate_fascia.withColumn('percentuale_prima_dose', dosi_somministrate_fascia.vaccinati_prima_dose / dosi_somministrate_fascia.popolazione * 100)
dosi_somministrate_fascia = dosi_somministrate_fascia.withColumn('percentuale_seconda_dose', dosi_somministrate_fascia.vaccinati_seconda_dose / dosi_somministrate_fascia.popolazione * 100)

dosi_somministrate_fascia.show(9)

+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+
fascia_anagrafica|vaccinati_prima_dose|vaccinati_seconda_dose|popolazione|percentuale_prima_dose|percentuale_seconda_dose|
+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+
 16-19| 22736| 7694| 2298846| 0.9890179681457566| 0.3346896660324354|
 20-29| 533077| 273143| 6084382| 8.761399267830324| 4.489248045241078|
 30-39| 779287| 364092| 6854632| 11.368764946097762| 5.311619938167359|
 40-49| 1133144| 463415| 8937229| 12.678918711828914| 5.185220161640705|
 50-59| 1546292| 629177| 9414195| 16.425111228310016| 6.683279876824306|
 60-69| 1789279| 500327| 7364364| 24.29644976809946| 6.793892860266006|
 70-79| 3062759| 396631| 5968373| 51.316481057735494| 6.6455464495935495|
 80-89| 3090756| 2123332| 3628160| 85.1879740695008| 58.52365937555124|
 90+| 683194| 482779| 791543| 86.31167226543599| 60.99213814031581|
+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+

### Percentuale popolazione vaccinata per regione e fascia anagrafica
Si riporta la percentuale di popolazione vaccinata (prima e seconda dose) per regione e fascia anagrafica.

In [0]:
dosi_somministrate = dati_vaccini_somministrazioni.groupBy('nome_area', 'fascia_anagrafica') \
                                                  .sum('prima_dose', 'seconda_dose') \
                                                  .orderBy('nome_area', 'fascia_anagrafica') \
                                                  .withColumnRenamed('nome_area', 'regione') \
                                                  .withColumnRenamed('sum(prima_dose)', 'vaccinati_prima_dose') \
                                                  .withColumnRenamed('sum(seconda_dose)', 'vaccinati_seconda_dose')

cond = [popolazione_regioni.denominazione_regione == dosi_somministrate.regione, popolazione_regioni.range_eta == dosi_somministrate.fascia_anagrafica]
dosi_somministrate = dosi_somministrate.join(popolazione_regioni.select('denominazione_regione', 'range_eta', 'totale_generale'), cond, how='left') \
                                       .drop('denominazione_regione', 'range_eta') \
                                       .withColumnRenamed('totale_generale', 'popolazione') \
                                       .orderBy('regione', 'fascia_anagrafica')

dosi_somministrate = dosi_somministrate.withColumn('percentuale_prima_dose', dosi_somministrate.vaccinati_prima_dose / dosi_somministrate.popolazione * 100)
dosi_somministrate = dosi_somministrate.withColumn('percentuale_seconda_dose', dosi_somministrate.vaccinati_seconda_dose / dosi_somministrate.popolazione * 100)

dosi_somministrate.show(189)